In [35]:
import pandas as pd
import numpy as np

# Загрузка даних
data = pd.read_csv('avocado.csv', delimiter=',', encoding='latin1')

# Видалення нульових значень
data = data.dropna()

# Вивід даних
data

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,7,2018-02-04,1.63,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,organic,2018,WestTexNewMexico
18245,8,2018-01-28,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,9,2018-01-21,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,10,2018-01-14,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Розділення класів даних
X = np.array(data[['AveragePrice', 'Small Bags']])
y = np.array(data['type'])

# Випадкові 100 рядків
random_indices = np.random.choice(len(data), 100, replace=False)
X = X[random_indices]
y = y[random_indices]

# Розбиття даних на навчальну та тестову групи, з відношенням 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Перемішування даних для оптимізації моделей з підходом перемішування
shuffled_indices = np.random.permutation(len(y_train))
X_train_shuffled = X_train[shuffled_indices]
y_train_shuffled = y_train[shuffled_indices]

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Створення моделі логістичної регресії
model = LogisticRegression()

# Навчання моделі на навчальних даних
model.fit(X_train, y_train)

# Передбачення класів для тестових даних
y_pred = model.predict(X_test)

# Оцінка точності та інших метрик
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print(f"Точність: {accuracy}")
print(f"Матриця плутанин:\n{confusion}")

Точність: 0.8666666666666667
Матриця плутанин:
[[ 7  3]
 [ 1 19]]


In [38]:
# Набір параметрів для оптимізації
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

In [39]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression

# Підбір оптимальних параметрів з допомогою методу Leave One Out

loo = LeaveOneOut()
best_score = 0
best_params = None

for C in param_grid['C']:
    model = LogisticRegression(C=C)

    scores = []

    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        scores.append(score)

    mean_score = np.mean(scores)
    
    # Перевірка на запис в змінну найкращого параметра
    if mean_score > best_score:
        best_score = mean_score
        best_params = {'C': C}

# Виведення найкращих параметрів
print("Найкращі параметри:", best_params)
print("Найкраща оцінка:", best_score)

Найкращі параметри: {'C': 0.1}
Найкраща оцінка: 0.89


In [40]:
from sklearn.model_selection import GridSearchCV, train_test_split

# Підбір параметрів на основі відкладеної+валідаційної вибірки з перемішуванням та без

# З перемішуванням
grid_search_shuffle = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search_shuffle.fit(X_train_shuffled, y_train_shuffled)

# Без перемішування
grid_search_noshuffle = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search_noshuffle.fit(X_train, y_train)

# Виведення найкращих параметрів для обох методів
print("Валідація з перемішуванням:")
print("Найкращі параметри:", grid_search_shuffle.best_params_)
print("Найкращий результат:", grid_search_shuffle.best_score_)

print("\nВалідація без перемішування:")
print("Найкращі параметри:", grid_search_noshuffle.best_params_)
print("Найкращий результат:", grid_search_noshuffle.best_score_)

# Оцінка кращої моделі на тестовій вибірці
best_model_shuffle = grid_search_shuffle.best_estimator_
best_model_noshuffle = grid_search_noshuffle.best_estimator_

accuracy_shuffle = best_model_shuffle.score(X_test, y_test)
accuracy_noshuffle = best_model_noshuffle.score(X_test, y_test)

print("\nОцінка кращої моделі на тестовій вибірці (з перемішуванням):", accuracy_shuffle)
print("Оцінка кращої моделі на тестовій вибірці (без перемішування):", accuracy_noshuffle)


Валідація з перемішуванням:
Найкращі параметри: {'C': 0.1}
Найкращий результат: 0.8

Валідація без перемішування:
Найкращі параметри: {'C': 0.01}
Найкращий результат: 0.7878947368421054

Оцінка кращої моделі на тестовій вибірці (з перемішуванням): 1.0
Оцінка кращої моделі на тестовій вибірці (без перемішування): 1.0


In [48]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split

# Підбір параметрів на основі випадкового та сіткового пошуку з перемішуванням та без

# Випадкове з перемішуванням
random_search_shuffle = RandomizedSearchCV(model, param_grid, n_iter=6, cv=5, scoring='accuracy')
random_search_shuffle.fit(X_train_shuffled, y_train_shuffled)

# Випадкове без перемішування
random_search_noshuffle = RandomizedSearchCV(model, param_grid, n_iter=6, cv=5, scoring='accuracy')
random_search_noshuffle.fit(X_train, y_train)

# Сіткове з перемішуванням
grid_search_shuffle = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search_shuffle.fit(X_train_shuffled, y_train_shuffled)

# Сіткове без перемішування
grid_search_noshuffle = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search_noshuffle.fit(X_train, y_train)

# Виведення найкращих параметрів для обох методів з випадковим пошуком
print("Випадковий пошук з перемішуванням:")
print("Найкращі параметри:", random_search_shuffle.best_params_)
print("Найкращий результат:", random_search_shuffle.best_score_)

print("\nВипадковий пошук без перемішування:")
print("Найкращі параметри:", random_search_noshuffle.best_params_)
print("Найкращий результат:", random_search_noshuffle.best_score_)

print()

# Виведення найкращих параметрів для обох методів з сітковим пошуком
print("Сітковий пошук з перемішуванням:")
print("Найкращі параметри:", grid_search_shuffle.best_params_)
print("Найкращий результат:", grid_search_shuffle.best_score_)

print("\nСітковий пошук без перемішування:")
print("Найкращі параметри:", grid_search_noshuffle.best_params_)
print("Найкращий результат:", grid_search_noshuffle.best_score_)

# Оцінка кращої моделі на тестовій вибірці з випадковим пошуком
random_best_model_shuffle = random_search_shuffle.best_estimator_
random_best_model_noshuffle = random_search_noshuffle.best_estimator_

random_accuracy_shuffle = random_best_model_shuffle.score(X_test, y_test)
random_accuracy_noshuffle = random_best_model_noshuffle.score(X_test, y_test)

print("\nОцінка кращої моделі на тестовій вибірці (випадковий пошук з перемішуванням):", random_accuracy_shuffle)
print("Оцінка кращої моделі на тестовій вибірці (випадковий пошук без перемішування):", random_accuracy_noshuffle)

# Оцінка кращої моделі на тестовій вибірці з сітковим пошуком
grid_best_model_shuffle = grid_search_shuffle.best_estimator_
grid_best_model_noshuffle = grid_search_noshuffle.best_estimator_

grid_accuracy_shuffle = grid_best_model_shuffle.score(X_test, y_test)
grid_accuracy_noshuffle = grid_best_model_noshuffle.score(X_test, y_test)

print("\nОцінка кращої моделі на тестовій вибірці (сітковий пошук з перемішуванням):", grid_accuracy_shuffle)
print("Оцінка кращої моделі на тестовій вибірці (сітковий пошук без перемішування):", grid_accuracy_noshuffle)

Випадковий пошук з перемішуванням:
Найкращі параметри: {'C': 0.1}
Найкращий результат: 0.8

Випадковий пошук без перемішування:
Найкращі параметри: {'C': 0.01}
Найкращий результат: 0.7878947368421054

Сітковий пошук з перемішуванням:
Найкращі параметри: {'C': 0.1}
Найкращий результат: 0.8

Сітковий пошук без перемішування:
Найкращі параметри: {'C': 0.01}
Найкращий результат: 0.7878947368421054

Оцінка кращої моделі на тестовій вибірці (випадковий пошук з перемішуванням): 1.0
Оцінка кращої моделі на тестовій вибірці (випадковий пошук без перемішування): 1.0

Оцінка кращої моделі на тестовій вибірці (сітковий пошук з перемішуванням): 1.0
Оцінка кращої моделі на тестовій вибірці (сітковий пошук без перемішування): 1.0


In [51]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedShuffleSplit

# Підбір параметрів на основі випадкового та сіткового пошуку зі стратифікованою вибіркою

# Випадковий пошук зі стратифікованою вибіркою
random_stratified_cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
random_search_stratified = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=6, cv=random_stratified_cv, n_jobs=-1)
random_search_stratified.fit(X_train, y_train)

# Сітковий пошук зі стратифікованою вибіркою
grid_stratified_cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid_search_stratified = GridSearchCV(model, param_grid, cv=grid_stratified_cv, n_jobs=-1)
grid_search_stratified.fit(X_train, y_train)

# Виведення найкращих параметрів та результатів
print("Найкращі параметри випадкового пошуку зі стратифікованою вибіркою:", random_search_stratified.best_params_)
print("Найкращий результат випадкового пошуку зі стратифікованою вибіркою:", random_search_stratified.best_score_)
print()
print("Найкращі параметри сіткового пошуку зі стратифікованою вибіркою:", grid_search_stratified.best_params_)
print("Найкращий результат сіткового пошуку зі стратифікованою вибіркою:", grid_search_stratified.best_score_)

Найкращі параметри випадкового пошуку зі стратифікованою вибіркою: {'C': 0.01}
Найкращий результат випадкового пошуку зі стратифікованою вибіркою: 0.7500000000000001

Найкращі параметри сіткового пошуку зі стратифікованою вибіркою: {'C': 0.01}
Найкращий результат сіткового пошуку зі стратифікованою вибіркою: 0.7500000000000001


In [52]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

# Підбір параметрів на основі випадкового та сіткового пошуку з випадковими перестановками (StratifiedKFold)

# Випадковий пошук з випадковими перестановками (StratifiedKFold)
random_stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
random_search_permutations = RandomizedSearchCV(model, param_grid, n_iter=6, cv=random_stratified_cv, n_jobs=-1)
random_search_permutations.fit(X_train, y_train)

# Сітковий пошук з випадковими перестановками (StratifiedKFold)
grid_stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search_permutations = RandomizedSearchCV(model, param_grid, n_iter=6, cv=grid_stratified_cv, n_jobs=-1)
grid_search_permutations.fit(X_train, y_train)

# Виведення найкращих параметрів та результатів
print("Випадковий пошук з випадковими перестановками:")
print("Найкращі параметри:", random_search_permutations.best_params_)
print("Найкращий результат:", random_search_permutations.best_score_)
print()
print("Сітковий пошук з випадковими перестановками:")
print("Найкращі параметри:", grid_search_permutations.best_params_)
print("Найкращий результат:", grid_search_permutations.best_score_)

Випадковий пошук з випадковими перестановками:
Найкращі параметри: {'C': 0.1}
Найкращий результат: 0.8089473684210526

Сітковий пошук з випадковими перестановками:
Найкращі параметри: {'C': 0.1}
Найкращий результат: 0.8089473684210526
